In [1]:
import vivisect
import Elf
import grpc

In [2]:
binary_path = "/home/davidanekstein/amp/Challenge-Problems-master/Challenge_03/build/program_c"
binary_path = "/home/davidanekstein/amp/main"

vw = vivisect.VivWorkspace()
vw.setMeta('Architecture', 'arm')
vw.setMeta('bigend', False)
vw.loadFromFile(binary_path)
vw.analyze()


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/davidanekstein/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/davidanekstein/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1043,

In [3]:
elf = Elf.elfFromFileName(binary_path)



--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/davidanekstein/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/davidanekstein/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1043,

In [4]:
emu = vw.getEmulator()
print(list(map(lambda x: vw.getName(x), vw.getEntryPoints())))
emu.setProgramCounter(vw.getEntryPoints()[0])
print(hex(elf.getBaseAddress()))

def step(n):
    for i in range(1, n):
        try:
            pc_before = emu.getProgramCounter()
            op = emu.parseOpcode(emu.getProgramCounter())
            emu.stepi()
            yield hex(emu.readMemValue(emu.getProgramCounter(), 4)), op, hex(op.opcode)
        except:
            pass

ops = step(20)
for op in ops:
    print(op)



['main._init', 'main._fini', 'main.frame_dummy', 'main.__do_global_dtors_aux', 'main.deregister_tm_clones', 'main.register_tm_clones', 'main.__weak___cxa_finalize@GLIBC_2.2.5', 'main.__entry', 'main.main']
0x2000000
('0x8ec8348', rep: nop edx, '0xf001')
('0xd9058b48', sub rsp,8, '0x2002')
('0x74c08548', mov rax,qword [rip + 12249], '0x6001')
('0xd0ff0274', test rax,rax, '0x5001')
('0x8348d0ff', jz 0x02001016, '0x1002')
('0x8c48348', call rax, '0x1003')
('0xc3', add rsp,8, '0x2001')
('0x61616161', ret , '0x1005')


In [6]:
# from concurrent import futures
import logging

import grpc
import vivserver_pb2
import vivserver_pb2_grpc


class Greeter(vivserver_pb2_grpc.EmulatorServicer):

    def RunNumInstructions(self, request, context):
        del request
        return vivserver_pb2_grpc.vivserver__pb2.RunInstructionsReply()


def serve():
    uds_addresses = ['unix:helloworld.sock', 'unix:///tmp/helloworld.sock']
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    vivserver_pb2_grpc.add_EmulatorServicer_to_server(Greeter(), server)
    for uds_address in uds_addresses:
        server.add_insecure_port(uds_address)
        logging.info('Server listening on: %s', uds_address)
    server.start()
    server.wait_for_termination()


if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO)
    serve()


INFO:root:Server listening on: unix:helloworld.sock
INFO:root:Server listening on: unix:///tmp/helloworld.sock


KeyboardInterrupt: 